# MNIST Classification with PyTorch
In this notebook, we will build and train a Convolutional Neural Network (CNN) to classify images from the MNIST dataset using PyTorch.
The notebook is structured to help understand the steps required to implement and train a model on a GPU.


In [1]:
# Step 1: Import Required Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# Ensure GPU is enabled in Colab
if torch.cuda.is_available():
    print('GPU is enabled.')
else:
    print('GPU is not available. Ensure you enable GPU runtime in Colab.')

GPU is enabled.


## Step 2: Define the CNN Model
The model consists of convolutional, dropout, and fully connected layers to process the MNIST dataset. Below is the model architecture.


In [2]:
# Define the CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## Step 3: Define Training and Testing Functions
The training and testing functions are used to train the model and evaluate its performance on the test set.


In [3]:
# Training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


# Testing function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

## Step 4: Load MNIST Dataset
We will use the MNIST dataset provided by the `torchvision` library. The dataset will be automatically downloaded if not already present.


In [4]:
# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

print(f'Training dataset size: {len(train_dataset)}')
print(f'Testing dataset size: {len(test_dataset)}')

Training dataset size: 60000
Testing dataset size: 10000


## Step 5: Training the Model
Now, we train the model for a specified number of epochs and evaluate its performance on the test set.


In [ ]:
# Initialize model, optimizer, and device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# Train and test the model
for epoch in range(1, 6):  # Train for 5 epochs
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

# Save the model
torch.save(model.state_dict(), 'mnist_cnn.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.330223
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.367531
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.535433
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.444740
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.549234
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.270578
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.286206
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.297153
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.276563
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.253177
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.303113
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.249876
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.162227
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.096275
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.177078
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.333757
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.210055
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.216158
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.158913
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.194153
Train Epoch: 1 [